In [1]:
# install Hugging Face Libraries
!pip install transformers peft datasets accelerate evaluate loralib --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [3]:
from datasets import load_dataset
dataset = load_dataset("squad")
print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['validation'])}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Train dataset size: 87599
Test dataset size: 10570


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-base"

# Load tokenizer of FLAN-t5
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from datasets import concatenate_datasets
import numpy as np

# Set a target maximum length (e.g., 512)
MAX_LENGTH = 512

# Function to preprocess inputs
def preprocess_inputs(examples):
    # Concatenate context and question as a single input string
    inputs = [context + " " + question for context, question in zip(examples["context"], examples["question"])]
    # Tokenize with truncation and padding
    return tokenizer(
        inputs,
        max_length=MAX_LENGTH,  # Limit the tokenized sequence length
        truncation=True,
        padding="max_length",  # Ensure consistent length by padding to MAX_LENGTH
    )

# Concatenate datasets and tokenize inputs
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["validation"]]).map(
    preprocess_inputs,
    batched=True,
    remove_columns=["context", "question", "answers"]  # Remove original columns to avoid conflict
)
input_lengths = [len(x) for x in tokenized_inputs["input_ids"]]
max_source_length = int(np.percentile(input_lengths, 85))
print(f"Max source length: {max_source_length}")

# Function to preprocess targets (answers)
def preprocess_targets(examples):
    # Extract the first answer from the `text` field, if available; otherwise, use an empty string
    answers = [ans["text"][0] if len(ans["text"]) > 0 else "" for ans in examples["answers"]]
    # Tokenize with truncation and padding
    return tokenizer(
        answers,
        max_length=MAX_LENGTH,  # Limit tokenized answer length
        truncation=True,
        padding="max_length",  # Ensure consistent length by padding to MAX_LENGTH
    )

# Tokenize targets
tokenized_targets = concatenate_datasets([dataset["train"], dataset["validation"]]).map(
    preprocess_targets,
    batched=True,
    remove_columns=["context", "question", "answers"]
)
target_lengths = [len(x) for x in tokenized_targets["input_ids"]]
max_target_length = int(np.percentile(target_lengths, 90))
print(f"Max target length: {max_target_length}")


Map:   0%|          | 0/98169 [00:00<?, ? examples/s]

Max source length: 512


Map:   0%|          | 0/98169 [00:00<?, ? examples/s]

Max target length: 512


In [7]:
def preprocess_function(sample, padding="max_length"):
    # Construct inputs by combining context and question
    inputs = [f"question: {q} context: {c}" for q, c in zip(sample["question"], sample["context"])]

    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Extract the first answer text as the target
    targets = [ans["text"][0] if len(ans["text"]) > 0 else "" for ans in sample["answers"]]

    # Tokenize targets
    labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

    # Replace `pad_token_id` with `-100` to ignore padding in the loss calculation
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    # Add labels to the inputs
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# def preprocess_function(sample, padding="max_length"):
#     # Construct inputs by combining context and question
#     inputs = [context + " " + question for context, question in zip(sample["context"], sample["question"])]

#     # Tokenize inputs
#     model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

#     # Extract the first answer text as the target
#     targets = [ans["text"][0] if len(ans["text"]) > 0 else "" for ans in sample["answers"]]

#     # Tokenize targets
#     labels = tokenizer(text_target=targets, max_length=max_target_length, padding=padding, truncation=True)

#     # Replace `pad_token_id` with `-100` to ignore padding in the loss calculation
#     if padding == "max_length":
#         labels["input_ids"] = [
#             [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
#         ]

#     # Add labels to the inputs
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# Map the function over the dataset
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["context", "question", "answers", "id"]  # Adjust columns for SQuAD
)

# Save tokenized datasets
tokenized_dataset["train"].save_to_disk("data/train")
tokenized_dataset["validation"].save_to_disk("data/eval")

print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/87599 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10570 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['title', 'input_ids', 'attention_mask', 'labels']


In [8]:
train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(10000))
val_dataset = tokenized_dataset["validation"].shuffle(seed=42).select(range(2000))

In [9]:
# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
from peft import LoraConfig, get_peft_model, TaskType

# Define LoRA Config
lora_config = LoraConfig(
 r=8,
 lora_alpha=16,
 target_modules=["q", "v"],
 lora_dropout=0.1,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# trainable params: 18874368 || all params: 11154206720 || trainable%: 0.16921300163961817

trainable params: 884,736 || all params: 248,462,592 || trainable%: 0.3561


In [11]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [12]:
import torch
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-lora-squad",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    fp16=False,
    gradient_accumulation_steps=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=5e-5,
    save_total_limit=1,
    predict_with_generate=True,
    report_to=[],  # <- disables wandb
    label_names=["labels"],
)
# training_args = Seq2SeqTrainingArguments(
#     output_dir=output_dir,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     learning_rate=1e-3, # higher learning rate
#     num_train_epochs=2,
#     # logging_dir=f"{output_dir}/logs",
#     logging_strategy="steps",
#     logging_steps=100,
#     save_strategy="epoch",
#     label_names=["labels"],
#     report_to=None,
# )

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    train_dataset=train_dataset)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [13]:
# train model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.407900,0.318747
2,0.370200,0.313855


TrainOutput(global_step=936, training_loss=0.38430471501798713, metrics={'train_runtime': 7970.2155, 'train_samples_per_second': 3.764, 'train_steps_per_second': 0.117, 'total_flos': 2.05692598222848e+16, 'train_loss': 0.38430471501798713, 'epoch': 2.992})

In [14]:
save_path = "/content/drive/MyDrive/T5/PEFT_FLANT5_SQUAD"
trainer.model.save_pretrained(save_path)